In [ ]:
!pip install pydicom
!pip install Pillow
!pip install dicom2jpg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00


In [ ]:
import dicom2jpg
import pydicom
from google.colab import drive
import os
import zipfile
import shutil
import re

drive.mount('/content/drive', force_remount=True)

input_path = '/content/drive/MyDrive/Có bệnh lý/'
output_path = '/content/drive/MyDrive/Dicom process/Có bệnh lý/'

def tinhten(name):
    name = re.sub(r'\b(M|F|T)\b', '', str(name))
    phanten = re.findall(r'\b[A-Za-zÀ-ỹ]+\b', name)
    return ' '.join(phanten)

def tojpg(dcm_path, export_path):
    dicom2jpg.dicom2jpg(dcm_path, target_root=export_path)
    count = 1
    for root, _, files in os.walk(export_path):
        for file in files:
            if file.endswith('.jpg'):
                src = os.path.join(root, file)
                dst = os.path.join(export_path, f"{count}.jpg")
                shutil.move(src, dst)
                count += 1
    for root, dirs, _ in os.walk(export_path):
        for dir in dirs:
            shutil.rmtree(os.path.join(root, dir))

for root, _, files in os.walk(input_path):
    for file in files:
        if file.endswith(".zip"):
            zip_path = os.path.join(root, file)
            raw_patient_name = file.split('_')[1]
            patient_name = tinhten(raw_patient_name)
            patient_output_path = os.path.join(output_path, patient_name)

            os.makedirs(patient_output_path, exist_ok=True)

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content/temp_dicom')

            for subdir, _, subfiles in os.walk('/content/temp_dicom'):
                for subfile in subfiles:
                    if subfile.endswith(".dcm"):
                        dcm_path = os.path.join(subdir, subfile)
                        ds = pydicom.dcmread(dcm_path)
                        series_description = ds.SeriesDescription if 'SeriesDescription' in ds else 'Unknown'
                        series_output_path = os.path.join(patient_output_path, series_description)

                        os.makedirs(series_output_path, exist_ok=True)

                        if hasattr(ds, 'PixelData'):
                            try:
                                tojpg(dcm_path, series_output_path)
                            except:
                                pass

            shutil.rmtree('/content/temp_dicom')

Mounted at /content/drive
Tệp DICOM không có dữ liệu pixel: /content/temp_dicom/1.3.46.670589.11.41238.5.0.7296.2020111815462008259/1.3.46.670589.11.41238.5.0.7296.2020111815463432265.dcm
Tệp DICOM không có dữ liệu pixel: /content/temp_dicom/1.3.46.670589.11.41238.5.0.7296.2020111815482910309/1.3.46.670589.11.41238.5.0.7296.2020111815495100332.dcm
Tệp DICOM không có dữ liệu pixel: /content/temp_dicom/1.3.46.670589.11.41238.5.0.8324.2020111815483201000/1.3.46.670589.11.41238.5.0.8324.2020111815483217001.dcm
Tệp DICOM không có dữ liệu pixel: /content/temp_dicom/1.3.46.670589.11.41238.5.0.14888.2020111815495411000/1.3.46.670589.11.41238.5.0.14888.2020111815495428003.dcm
Tệp DICOM không có dữ liệu pixel: /content/temp_dicom/1.3.46.670589.11.41238.5.0.7296.2020111815480552288/1.3.46.670589.11.41238.5.0.7296.2020111815480560289.dcm
Tệp DICOM không có dữ liệu pixel: /content/temp_dicom/1.3.46.670589.11.41238.5.0.7296.2020111815523099469/1.3.46.670589.11.41238.5.0.7296.2020111815540762590.dcm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')